# Digital Galton Board (Raspberry Pi Plinko)
#### [V. Hunter Adams](https://vanhunteradams.com)

## Introduction

In this laboratory exercise, you will engineer a digital version of one of the most famous physical demonstrations of order from chaos that has ever been constructed: the Galton Board.

A Galton Board is composed of interleaved rows of pegs. When one drops a ball onto the top row of pegs, it bounces its way to the bottom and ultimately lands in one of N bins that rest beneath the N lowest pegs in the board. In an idealized Galton Board, the ball bounces against a peg in each row of the board. When it strikes that peg, it has a 50% chance of bouncing to the left, and a 50% chance of bouncing to the right. The number of paths to the right-most bin at the bottom of the board is 1 (the ball had to bounce to the right in every row of the board). The number of paths to the first bin in from the right is equal to the number of rows in the board (the ball had to bounce right on every row except for one, and it doesn't matter on which one it bounced left). And indeed the path to the kth row in from the right is the binomial coefficient $\left(\begin{matrix}n\\k\end{matrix}\right)$, where $n$ is the number of rows and $k$ is the count (from either side) of the bin. The upshot of all of this is that if you repeat this experiment over and over and over again, the count of balls in the bins forms a nice clean binomial distribution. And as the number of rows increases and the number of trials increases, this binomial distribution, by the central limit theorem, approaches a Gaussian.

It is startling when you see it, it looks like order emerging from absolute chaos. In fact, this phenomenon so impressed Francis Galton that he stated "The law [the central limit theorem] would have been personified by the Greeks and deified, if they had known of it. It reigns with serenity and in complete self-effacement amidst the wildest confusion. The huger the mob, and the greater the apparent anarchy, the more perfect is its sway. It is the supreme law of Unreason. Whenever a large sample of chaotic elements are taken in hand and marshalled in the order of their magnitude, an unsuspected and most beautiful form of regularity proves to have been latent all along."

In this lab, you will build a digital version of a Board, through which you will drop as many balls as you can manage at 30 frames per second. The more thoughtful and efficient your code, the more balls you will be able to animate. Every time any ball strikes a peg there must be a "thunk!" sound effect, and you must count and plot the number of balls that land in each bin at the bottom of the board. Furthermore, you will implement a playable game based on this board. The user will control the position of a paddle by means of a potentiometer, with which they will try to catch as many balls as they can.